<font size="5">Ingest Time Series - GHG total without LULUCF, in kt CO₂ equivalent into Trino pipeline</font>


Load Environment Variables

In [1]:
from pcaf_env import *
import ParseXLS as parser

ingest_table = 'sf_unfccc_without_lulucf'
models_table = 'pcaf_dbt_models'

using connect string: trino://MichaelTiemannOSC@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev
create schema if not exists osc_datacommons_dev.mdt_sandbox


In [2]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('aicoe_osc_demo_results',)
('default',)
('demo_dv',)
('dera',)
('essd',)
('iceberg_demo',)
('information_schema',)
('ingest',)
('mdt_sandbox',)
('pcaf_sovereign_footprint',)
('rmi',)
('sandbox',)
('wri_gppd',)


Load PCAF_UNFCC_WITHOUT_LULUCF

In [4]:
ghg_without_lulu_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/UNFCCC/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx')
ghg_without_lulu_file.download_file(f'/tmp/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx')

df = parser.process('UNFCCC_without_LULUCF.ini','UNFCCC_without_LULUCF.csv') 

co2_removals_excl_lulu_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/UNFCCC/Annual_Net_emissions_removals_in_Gg_CO₂_equivalent_excl_LULUCF.csv')
co2_removals_excl_lulu_file.download_file(f'/tmp/Annual_Net_emissions_removals_in_Gg_CO₂_equivalent_excl_LULUCF.csv')

df2 = parser.process('UNFCCC_without_LULUCF_other.ini','UNFCCC_without_LULUCF_other.csv') 

# combine both dataframes

df = pd.concat([df,df2])

#pd.to_numeric(df["validity_date"],errors='raise')
df = df.astype({'validity_date': 'int32', 'country_iso_code': 'string', 'value': 'float'})

df = df.convert_dtypes()
df.info(verbose=True)
df = df[['rec_source','data_provider','country_iso_code','country_name','validity_date','attribute','value','value_units']].dropna(subset=['value'])






European Union (Convention) not found in regex
European Union (KP) not found in regex


UNFCCC_without_LULUCF.ini
file_list:
['/tmp/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx']
/tmp/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx
2
xls
/tmp/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalen
['0', '2']
eval_components
['Time Series - GHG total without LULUCF, in kt CO₂ equivalent']
['Party', 'Base year', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', 'Last Inventory Year (2020)', 'Change from base year to latest reported year']
Time Series - GHG total without LULUCF, in kt CO₂ equivalent
eval_components
['kt CO2e']
['Party', 'Base year', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '

Party \ Unit not found in regex


UNFCCC_without_LULUCF_other.ini
file_list:
['/tmp/Annual_Net_emissions_removals_in_Gg_CO₂_equivalent_excl_LULUCF.csv']
/tmp/Annual_Net_emissions_removals_in_Gg_CO₂_equivalent_excl_LULUCF.csv
2
csv
/tmp/Annual_Net_emissions_removals_in_Gg_CO₂_equivalent_excl_LULUC
                                   Year               1990               1991   
0                          Party \ Unit  Gg CO₂ equivalent  Gg CO₂ equivalent  \
1                           Afghanistan                NaN                NaN   
2                               Albania            4341.63            6486.66   
3                               Algeria                NaN                NaN   
4                                Angola                NaN                NaN   
..                                  ...                ...                ...   
144  Venezuela (Bolivarian Republic of)                NaN                NaN   
145                            Viet Nam                NaN                NaN   
146    

In [6]:
try:
    os.mkdir("/opt/app-root/src/PCAF-sovereign-footprint/dbt/pcaf_transform/models", mode=0o755)
except FileExistsError:
    pass
shutil.rmtree("/opt/app-root/src/PCAF-sovereign-footprint/dbt/pcaf_transform/models/.ipynb_checkpoints", ignore_errors=True)

In [13]:
custom_meta_content = {
    'data provider': df[['data_provider']],
    'title': 'UNFCCC GHG Total excluding LULUCF by country and year',
    'author': 'UNFCCC',
    'contact': 'GHGdata@unfccc.int',
    'description': json.dumps("""
UNFCCC GHG Total excluding LULUCF by country and year"""),
    'release_date': '2022-01-01 00:00:00',
    # How should we describe our transformative step here?
}

def description_is (s):
    return { 'Description': json.dumps(s)}

custom_meta_fields = {}
custom_meta_fields['rec_source'] = description_is("Annual_Net_emissions_removals_in_Gg_CO₂_equivalent_excl_LULUCF.csv and Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx")
custom_meta_fields['data_provider'] = description_is("UNFCCC")
custom_meta_fields['country_iso_code'] = description_is("ISO-3166 Country Code (alpha_3)")
custom_meta_fields['country_name'] = description_is("ISO-3166 Country Name")
custom_meta_fields['attribute'] = description_is("Time Series - GHG total with LULUCF")
custom_meta_fields['value'] = description_is("kt CO₂ equivalent")
custom_meta_fields['value_units'] = description_is("kt CO2e")
custom_meta_fields['validity_date'] = description_is("Year of measurement")

for f in custom_meta_fields:
    if 'ISO' in custom_meta_fields[f]['Description']:
        custom_meta_fields[f]['tags'] = ['ISO']
    elif 'IPCC' in custom_meta_fields[f]['Description']:
        custom_meta_fields[f]['tags'] = ['IPCC']
    elif 'WDI' in custom_meta_fields[f]['Description']:
        custom_meta_fields[f]['tags'] = ['WDI']
    elif f == 'year':
        custom_meta_fields[f]['tags'] = ['annual']

rec_source
data_provider
country_iso_code
country_name
attribute
value
value_units
validity_date


In [5]:
create_trino_table_and_dbt_metadata(ingest_table, df, ['country_iso_code'], custom_meta_content, custom_meta_fields, verbose=True)

drop table if exists mdt_sandbox.sf_unfccc_without_lulucf_source
enforcing dataframe partition column order

verifying existence of table osc_datacommons_dev.mdt_sandbox.sf_unfccc_without_lulucf_source
create table if not exists osc_datacommons_dev.mdt_sandbox.sf_unfccc_without_lulucf_source (
    rec_source varchar,
    data_provider varchar,
    country_name varchar,
    validity_date integer,
    attribute varchar,
    value double,
    value_units varchar,
    country_iso_code varchar
) with (
    format = 'parquet',
    partitioning = array['country_iso_code']
)

staging dataframe parquet to s3 osc-datacommons-s3-bucket-dev02
/tmp/ingest_temp_59bcbef0/country_iso_code=AUS/0d5d44f8dd404a73ab404a604c166251-0.parquet  -->  trino/ingest/ingest_temp_59bcbef0/country_iso_code=AUS/0d5d44f8dd404a73ab404a604c166251-0.parquet
/tmp/ingest_temp_59bcbef0/country_iso_code=BLR/0d5d44f8dd404a73ab404a604c166251-0.parquet  -->  trino/ingest/ingest_temp_59bcbef0/country_iso_code=BLR/0d5d44f8dd404a73

In [6]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}_source
where country_iso_code='CAN' and validity_date=2020"""
pd.read_sql(sql, engine)


,rec_source,data_provider,country_name,validity_date,attribute,value,value_units,country_iso_code
0,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCCC,Canada,2020,"Time Series - GHG total without LULUCF, in kt ...",672354.02196,kt CO2e,CAN


In [16]:
osc._do_sql(f"insert into {ingest_schema}.{models_table} values(\'{json.dumps(dbt_dict['models'])}\')", engine, verbose=True)

insert into mdt_sandbox.pcaf_dbt_models values('{"sf_wdi_gdp": {"description": "\"\\nThis indicator provides per capita values for gross domestic product (GDP) expressed in current international dollars converted by purchasing power parity (PPP) conversion factor. \\n\\nGDP is the sum of gross value added by all resident producers in the country plus any product taxes and minus any subsidies not included in the value of the products. conversion factor is a spatial price deflator and currency converter that controls for price level differences between countries. Total population is a mid-year population based on the de facto definition of population, which counts all residents regardless of legal status or citizenship.\"", "columns": {"rec_source": {"description": "\"API_NY.GDP.MKTP.CD_DS2_en_csv_v2.csv and API_NY.GDP.MKTP.PP.CD_DS2_en_csv_v2.csv\""}, "data_provider": {"description": "\"WDI\"", "tags": ["WDI"]}, "country_iso_code": {"description": "\"ISO-3166 Country Code (alpha_3)\"", 

[(1,)]